In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from typing import Any, Dict, List

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import HandoffTermination, TextMentionTermination
from autogen_agentchat.messages import HandoffMessage
from autogen_agentchat.teams import Swarm
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
def refund_flight(flight_id: str) -> str:
    """Refund a flight"""
    return f"Flight {flight_id} refunded"

In [4]:
model_client = OpenAIChatCompletionClient(
    model="gpt-4o-mini",
    # api_key="YOUR_API_KEY",
)

travel_agent = AssistantAgent(
    "travel_agent",
    model_client=model_client,
    handoffs=["flights_refunder", "user"],
    system_message="""You are a travel agent.
    The flights_refunder is in charge of refunding flights.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    Use TERMINATE when the travel planning is complete.""",
)

flights_refunder = AssistantAgent(
    "flights_refunder",
    model_client=model_client,
    handoffs=["travel_agent", "user"],
    tools=[refund_flight],
    system_message="""You are an agent specialized in refunding flights.
    You only need flight reference numbers to refund a flight.
    You have the ability to refund a flight using the refund_flight tool.
    If you need information from the user, you must first send your message, then you can handoff to the user.
    When the transaction is complete, handoff to the travel agent to finalize.""",
)

In [5]:
termination = HandoffTermination(target="user") | TextMentionTermination("TERMINATE")
team = Swarm([travel_agent, flights_refunder], termination_condition=termination)

In [6]:
task = "I need to refund my flight."


async def run_team_stream() -> None:
    task_result = await Console(team.run_stream(task=task))
    last_message = task_result.messages[-1]

    while isinstance(last_message, HandoffMessage) and last_message.target == "user":
        user_message = input("User: ")

        task_result = await Console(
            team.run_stream(task=HandoffMessage(source="user", target=last_message.source, content=user_message))
        )
        last_message = task_result.messages[-1]


# Use asyncio.run(...) if you are running this in a script.
await run_team_stream()

---------- user ----------
I need to refund my flight.
---------- travel_agent ----------
I can help you with that! Please provide me with your flight details, including the booking reference number, the name on the ticket, and the reason for the refund, so I can assist you further.
---------- travel_agent ----------
[FunctionCall(id='call_otAF6Wp0QVa4QvrFWPAVPOgf', arguments='{}', name='transfer_to_user')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to user, adopting the role of user immediately.', call_id='call_otAF6Wp0QVa4QvrFWPAVPOgf')]
---------- travel_agent ----------
Transferred to user, adopting the role of user immediately.


User:  my Flight id 5


---------- user ----------
my Flight id 5
---------- travel_agent ----------
[FunctionCall(id='call_5YXHuSwvdV9YH4KyuO6RPM2i', arguments='{}', name='transfer_to_flights_refunder')]
---------- travel_agent ----------
[FunctionExecutionResult(content='Transferred to flights_refunder, adopting the role of flights_refunder immediately.', call_id='call_5YXHuSwvdV9YH4KyuO6RPM2i')]
---------- travel_agent ----------
Transferred to flights_refunder, adopting the role of flights_refunder immediately.
---------- flights_refunder ----------
[FunctionCall(id='call_Pmqn9ecNjalzeGd69BZqdSm3', arguments='{"flight_id":"5"}', name='refund_flight')]
---------- flights_refunder ----------
[FunctionExecutionResult(content='Flight 5 refunded', call_id='call_Pmqn9ecNjalzeGd69BZqdSm3')]
---------- flights_refunder ----------
Flight 5 refunded
---------- flights_refunder ----------
[FunctionCall(id='call_QgEar9TjcFs5cXh8xbEcxow1', arguments='{}', name='transfer_to_travel_agent')]
---------- flights_refunder -

User:  TERMINATE


---------- user ----------
TERMINATE
